In [ ]:
# Import the packages

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM
# Import my function
from Normalization import Normalization


In [ ]:
# Options
mini_batch_size = 2880
Delay = 8
time_horizon = 1
Training_MSE = []
Training_MAE = []
The_whole_prediction = []
The_whole_actual = []
pd.set_option("display.precision", 3)

In [ ]:
# Read and prepare the data
df = pd.read_excel('speed_volume_test.xlsx')
data = pd.DataFrame(data=df)
# data.info()

# select the target time series
target = ['Flow']
features = data.drop(['Date'], axis=1).columns

X_old = data[features].values
T_old = data[target].values

# Normalized the data based on the normalization function
normalizedX = Normalization(X_old, type_of_normalization = 1) # Range from 1 to 4
normalizedT = Normalization(T_old, type_of_normalization = 1)

In [ ]:
# Initialize the input and target series
X, T = [], []
for k in range(len(normalizedT)-Delay-1):
    a = normalizedX[k:(k+Delay), :]
    flatted = a.flatten()
    X.append(flatted) 
    T.append(normalizedT[k + Delay]) 

X = np.array(X)
T = np.array(T)

In [ ]:
# split the data for create the model and the rest
training_input = X[:mini_batch_size, :]
training_output = T[:mini_batch_size, :]
rest_of_input = X[mini_batch_size:-1, :]
rest_of_output = T[mini_batch_size:-1, :]
# convert to LSTM friendly format
inputs = training_input.reshape((training_input.shape[0], training_input.shape[1], 1))

In [ ]:
history_loss = []
# design network
model = Sequential()
model.add(LSTM(20, input_shape=(inputs.shape[1], inputs.shape[2])))
# model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(inputs, training_output, epochs=500, verbose=1, shuffle=False)
history_loss = history_loss + model.history.history['loss']

In [ ]:
# Get pair of (X, y) of the current minibatch/chunk
for i in range(0, rest_of_input.shape[0], time_horizon):

    X_train_mini = rest_of_input[i:i + time_horizon]
    y_train_mini = rest_of_output[i:i + time_horizon]

    prediction = []
    for j in range(len(X_train_mini)):
        testX = X_train_mini[j]
        X_train_mini_reshaped = testX.reshape((1, testX.shape[0], 1))
        Predicted = model.predict(X_train_mini_reshaped)
        prediction = np.concatenate((prediction, Predicted.ravel()))

    denormalization_actual = T_old.min() + (T_old.max() - T_old.min())*y_train_mini
    denormalization_predict = T_old.min() + (T_old.max() - T_old.min())*prediction
    MSE = mean_squared_error(denormalization_actual, denormalization_predict)
    MAE = np.sum(np.abs(denormalization_actual -
                        denormalization_predict))/time_horizon
    Training_MSE.append(MSE)
    Training_MAE.append(MAE)
    The_whole_prediction = np.concatenate(
        (The_whole_prediction, denormalization_predict.ravel()))
    The_whole_actual = np.concatenate(
        (The_whole_actual, denormalization_actual.ravel()))

    # Update the training input and output
    training_input = np.vstack((training_input[time_horizon:], X_train_mini))
    training_output = np.vstack((training_output[time_horizon:], y_train_mini))

    # convert to LSTM friendly format
    inputs = training_input.reshape(
        (training_input.shape[0], training_input.shape[1], 1))

    model.fit(inputs, training_output, epochs=500,
              verbose=1, shuffle=False)

    # save the loss of this epoch to history_loss
    # history_loss.append(model.history.history['loss'])
    history_loss = history_loss + model.history.history['loss']

In [ ]:
# Set the figure size in inches
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Plot the predicted and actual values
ax1.plot(The_whole_actual, label='Actual')
ax1.plot(The_whole_prediction, label='Predicted')
ax1.set_title('Comparison of Predicted and Actual Values')
ax1.set_xlabel('Time')
ax1.set_ylabel('Flow')
ax1.legend()

# Plot the training loss history
ax2.plot(history_loss)
ax2.set_title('Training Loss History')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')

# Save the combined figure as a JPEG file
# plt.savefig('combined_plots.jpg', bbox_inches='tight')
plt.show()
